# Weather Derivatives for Risk Management
**Pricing a Cooling Degree Days (CDD) Option using Real Weather Data**

This notebook models and prices a temperature-based derivative that helps companies hedge against weather-related risks.
We estimate the fair price of a monthly Cooling Degree Days (CDD) option using historical temperature data and Monte Carlo simulation.

In [ ]:

# === Imports ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from meteostat import Stations, Daily
from statsmodels.tsa.ar_model import AutoReg
import os

plt.style.use('seaborn-v0_8-whitegrid')

# === Parameters ===
station_id = 'USW00014933'  # Des Moines, Iowa
start = datetime(2000, 1, 1)
end = datetime(2024, 12, 31)
base_temp = 65.0
month_to_price = 7  # July
notional = 100.0
strike = 200.0
n_sim = 5000
seed = 42
np.random.seed(seed)


## Step 1. Fetch Daily Temperature Data

In [ ]:

# Fetch daily data from Meteostat
daily = Daily(station_id, start, end).fetch()

if daily['tavg'].isnull().all():
    daily['tavg'] = (daily['tmax'] + daily['tmin']) / 2

daily = daily.dropna(subset=['tavg'])
daily['tavg_F'] = daily['tavg'] * 9 / 5 + 32  # convert °C to °F

daily['cdd'] = (daily['tavg_F'] - base_temp).clip(lower=0)
monthly_cdd = daily.resample('M')['cdd'].sum()
monthly_cdd_month = monthly_cdd[monthly_cdd.index.month == month_to_price]

monthly_cdd_month.plot(marker='o', title=f'Historical Monthly CDD (Month={month_to_price})')
plt.ylabel('Cooling Degree Days')
plt.show()


## Step 2. Fit Seasonal + AR(1) Temperature Model

In [ ]:

# Prepare design matrix for seasonality
daily['dayofyear'] = daily.index.dayofyear
X = np.column_stack([
    np.ones(len(daily)),
    np.sin(2 * np.pi * daily['dayofyear'] / 365.25),
    np.cos(2 * np.pi * daily['dayofyear'] / 365.25)
])
y = daily['tavg_F'].values

# Fit linear regression
beta = np.linalg.lstsq(X, y, rcond=None)[0]
daily['seasonal'] = X.dot(beta)
daily['resid'] = daily['tavg_F'] - daily['seasonal']

# Fit AR(1) on residuals
ar_model = AutoReg(daily['resid'].dropna(), lags=1, old_names=False).fit()
phi = ar_model.params.get('lag1', ar_model.params.iloc[1])
sigma = np.std(ar_model.resid)

print(f"AR(1) coefficient phi = {phi:.3f}, sigma = {sigma:.3f}")


## Step 3. Monte Carlo Simulation for Monthly CDDs

In [ ]:

from tqdm import trange

# Day-of-year indices for target month (July)
example_year = 2024
month_days = pd.date_range(datetime(example_year, month_to_price, 1), 
                           datetime(example_year, month_to_price, 31))
doys = month_days.dayofyear.values

sim_totals = np.zeros(n_sim)

for i in trange(n_sim):
    resid = np.zeros(len(doys))
    resid[0] = np.random.normal(0, sigma / np.sqrt(1 - phi**2))
    for t in range(1, len(doys)):
        resid[t] = phi * resid[t-1] + sigma * np.random.normal()
    temps = (beta[0] + beta[1] * np.sin(2 * np.pi * doys / 365.25) +
             beta[2] * np.cos(2 * np.pi * doys / 365.25)) + resid
    cdd_total = np.clip(temps - base_temp, 0).sum()
    sim_totals[i] = cdd_total

# Compute payouts
payouts = np.maximum(sim_totals - strike, 0) * notional
price_estimate = payouts.mean()
price_std = payouts.std()
print(f"Estimated option price = ${price_estimate:,.2f} ± {price_std:,.2f}")


## Step 4. Visualize and Save Results

In [ ]:

os.makedirs("figures", exist_ok=True)

plt.figure(figsize=(8, 5))
plt.hist(sim_totals, bins=40, alpha=0.7, edgecolor='black')
plt.axvline(strike, color='red', linestyle='--', label=f'Strike = {strike}')
plt.title(f"Simulated Distribution of Monthly CDDs (Month={month_to_price})")
plt.xlabel("Monthly Cooling Degree Days (CDD)")
plt.ylabel("Frequency")
plt.legend()
plt.grid(alpha=0.3)

plt.savefig("figures/cdd_histogram.png", dpi=300, bbox_inches='tight')
plt.show()


## Step 5. Conclusions
- The simulation produces a realistic distribution of monthly Cooling Degree Days.
- The estimated option price provides an intuitive way to value weather derivatives.
- This framework can be extended to include climate trends or non-Gaussian residuals.